# Import libraries

In [ ]:
import os
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from PIL import Image
import cv2
import glob
#--------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


# Import Dataset

In [ ]:
!wget http://files.fast.ai/data/dogscats.zip # a folder with cat and dog images

--2021-09-07 12:57:15--  http://files.fast.ai/data/dogscats.zip
Resolving files.fast.ai (files.fast.ai)... 172.67.69.159, 104.26.2.19, 104.26.3.19, ...
Connecting to files.fast.ai (files.fast.ai)|172.67.69.159|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.fast.ai/data/dogscats.zip [following]
--2021-09-07 12:57:15--  https://files.fast.ai/data/dogscats.zip
Connecting to files.fast.ai (files.fast.ai)|172.67.69.159|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-09-07 12:57:15 ERROR 404: Not Found.



In [ ]:
!unzip dogscats.zip

unzip:  cannot find or open dogscats.zip, dogscats.zip.zip or dogscats.zip.ZIP.


In [ ]:
PATH = "dogscats/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = '/content/drive/MyDrive/AI_SIC_PROJECT/DataSets/day/'  #change dir to your project folder
files = glob.glob(PATH+'fire_images/*')
cat_train = [fn for fn in files]
files = glob.glob(PATH+'nonfire_images/*')
dog_train = [fn for fn in files]
len(cat_train), len(dog_train)

(0, 0)

In [ ]:
# files = glob.glob(PATH+'train/cats/*')
# cat_train = [fn for fn in files if 'cat' in fn]
# files = glob.glob(PATH+'train/dogs/*')
# dog_train = [fn for fn in files if 'dog' in fn]
# len(cat_train), len(dog_train)

(0, 0)

In [ ]:
PATH = '/content/drive/MyDrive/AI_SIC_PROJECT/DataSets/night/'  #change dir to your project folder
files = glob.glob(PATH+'fire/*')
cat_val = [fn for fn in files]
files = glob.glob(PATH+'nonfire/*')
dog_val = [fn for fn in files]
len(cat_val), len(dog_val)

(0, 0)

In [ ]:
# files = glob.glob(PATH+'valid/cats/*')
# cat_val = [fn for fn in files if 'cat' in fn]
# files = glob.glob(PATH+'valid/dogs/*')
# dog_val = [fn for fn in files if 'dog' in fn]
# len(cat_val), len(dog_val)


(0, 0)

In [ ]:
train_files = np.concatenate([cat_train[:200], dog_train[:200]]) # For technical issues we will only use 2000 images for each class
validation_files = np.concatenate([cat_val, dog_val])

![Texte alternatif…](https://faunalytics.org/wp-content/uploads/2017/11/14017928228_56f72a1aeb_krecrop-690x325.jpg)

# Prepare dataset

In [ ]:
IMG_DIM = (224, 224)

train_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in train_files]
train_imgs = np.array(train_imgs)
train_labels = [fn.split('/')[3].split('.')[0].strip() for fn in train_files]


In [ ]:
validation_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in validation_files]
validation_imgs = np.array(validation_imgs)
validation_labels = [fn.split('/')[3].split('.')[0].strip() for fn in validation_files]

In [ ]:
print('Train dataset shape:', train_imgs.shape, 
      '\tValidation dataset shape:', validation_imgs.shape)

Train dataset shape: (0,) 	Validation dataset shape: (0,)


In [ ]:
le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
validation_labels_enc = le.transform(validation_labels)

In [ ]:
# For CNN model (1) we will need rescaled images
import os
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from PIL import Image
import cv2
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#--------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

train_set = ImageDataGenerator(rescale=1./255).flow(train_imgs, train_labels_enc, batch_size=30)

val_set = ImageDataGenerator(rescale=1./255).flow(validation_imgs, validation_labels_enc, batch_size=20)

ValueError: ignored

In [ ]:
# To improve performance in the second part we will be using VGG model + data augmentation 

train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow(train_imgs, train_labels_enc, batch_size=30)
val_generator = val_datagen.flow(validation_imgs, validation_labels_enc, batch_size=20)

In [ ]:
# See output of data augmentation 
cat_generator = train_datagen.flow(train_imgs[1120:1121], train_labels[1120:1121],
                                   batch_size=1)
cat = [next(cat_generator) for i in range(0,5)]
fig, ax = plt.subplots(1,5, figsize=(16, 6))
print('Labels:', [item[1][0] for item in cat])
l = [ax[i].imshow(cat[i][0][0]) for i in range(0,5)]